<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# LinkedIn - Generate Leads from likes
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/LinkedIn/LinkedIn_Get_comments_from_post.ipynb" target="_parent"><img src="https://img.shields.io/badge/-Open%20in%20Naas-success?labelColor=000000&logo="/></a>

**Tags:** #linkedin #post #comments #naas_drivers

## Input

### Import libraries

In [5]:
from naas_drivers import linkedin, hubspot
import pandas as pd
import numpy as np
import naas
from datetime import datetime, timedelta
import requests
import json

### Setup your LinkedIn
- Get your cookies : <a href='https://www.notion.so/LinkedIn-driver-Get-your-cookies-d20a8e7e508e42af8a5b52e33f3dba75'>How to get your cookies ?</a>

In [6]:
# Get your cookies
LI_AT = ''  # EXAMPLE AQFAzQN_PLPR4wAAAXc-FCKmgiMit5FLdY1af3-2
JSESSIONID = ''  # EXAMPLE ajax:8379907400220387585

# Enter your profile URL
PROFILE_URL = "https://www.linkedin.com/in/florent-ravenel/"

### Setup your Hubspot
- Get your API token
- Get your hubspot owner ID
- Get your hubspot company ID

In [7]:
#initial parameters for hubspot connection
HS_CONTACT_ASSIGN_TO_ID = "" #contact id to whom task needs to be assigned
HS_API_TOKEN = ""
HUB_COMPANY_ID = ""

## Model

### Get posts from feed

In [5]:
df_posts = linkedin.connect(LI_AT, JSESSIONID).profile.get_posts_feed(PROFILE_URL, count=100, limit=-1)

### Get likes from posts

In [7]:
def get_likes(df):
    df_likes = pd.DataFrame()
    for index, row in df.iterrows():
        post_url = row['POST_URL']
        tmp_df = linkedin.connect(LI_AT, JSESSIONID).post.get_likes(post_url)
        if not tmp_df.empty:
            df_likes = pd.concat([df_likes, tmp_df])
    return df_likes
        
df_likes = get_likes(df_posts)
df_likes

- Check the links with <b>PROFILE_ID</b> for users posts. (i.e. Here activity posts are removed)
- Ignore below line if you want to consider all types of posts

In [8]:
# profile_posts = DF_all_post_likes[DF_all_post_likes['POST_URL'].str.contains(PROFILE_ID)]

In [10]:
# grouped_profile_posts = profile_posts.groupby(['PROFILE_ID']).size().sort_values(ascending=False).reset_index(name='count')

- Grouping the like count and classify the leads as per levels i.e. <b>L1, L2, L3, L4.</b> ( from highest to lowest counts of likes) with column name <b>"POTENTIAL LEAD"</b>
- Group posts as per the the column <b>"POTENTIAL LEAD"</b>

In [12]:
conditions = [
    (grouped_profile_posts['count'] <= 2),
    (grouped_profile_posts['count'] > 2) & (grouped_profile_posts['count'] <= 9),
    (grouped_profile_posts['count'] > 9) & (grouped_profile_posts['count'] <= 15),
    (grouped_profile_posts['count'] > 15)
    ]

values = ['L4', 'L3', 'L2', 'L1']

In [13]:
grouped_profile_posts['POTENTIAL_LEAD'] = np.select(conditions, values)

### Get profile details based on the POST_URL above
- filter rows from "grouped_profile_posts" dataframe based on L1, L2, & L3 levels and add data to leads_list
- Extract EMAIL, FIRSTNAME, LASTNAME, PHONE_NUMBER, OCCUPATION from profile URL

In [18]:
leads_list = [];

r_count = 1
for index, row in grouped_profile_posts.iterrows():
    if row['POTENTIAL_LEAD'] == "L3" or row['POTENTIAL_LEAD'] == "L2" or row['POTENTIAL_LEAD'] == "L1":
        profileid = row['PROFILE_ID']
        profileurl = "https://www.linkedin.com/in/"+profileid+"/"
       
        PROFILECONTACTS = linkedin.connect(LI_AT, JSESSIONID).profile.get_contact(profileurl)
        PROFILEIDENTITY = linkedin.connect(LI_AT, JSESSIONID).profile.get_identity(profileurl)
        
        profileemail = PROFILECONTACTS.at[0,'EMAIL']
        profilephoneno = PROFILECONTACTS.at[0,'PHONENUMBER']
        profilename = PROFILEIDENTITY.at[0,'FIRSTNAME'] + " "+ PROFILEIDENTITY.at[0,'LASTNAME']
        profilefirstname = PROFILEIDENTITY.at[0,'FIRSTNAME']
        profilelastname = PROFILEIDENTITY.at[0,'LASTNAME']
        profileoccupation = PROFILEIDENTITY.at[0,'OCCUPATION']
        if profileemail != None :
            leads_list.append([r_count,profilename,profilefirstname,profilelastname,profileemail,profilephoneno,profileoccupation,profileurl])
            r_count = r_count + 1
   

### Create hubspot contacts from linkedin likes

In [1]:
#to store the resulting output of create contact method
huboutput = ""

<h4>HS_CONTACT_ID_LIST</h4> - The list to store the contact id recently created and the linkedin profile URL of associated contact

In [20]:
HS_CONTACT_ID_LIST = []
for i in leads_list:
        profilename = i[1]
        profilefirstname = i[2]
        profilelastname = i[3]
        profileemail = i[4]
        profilephoneno = i[5]
        profileoccupation = i[6]
        profileurl = i[7]
        # With send method
        data = {"properties": 
                {
                    "linkedinbio": profileurl,
                    "firstname": profilefirstname,
                    "lastname": profilelastname,
                    "jobtitle": profileoccupation,
                    "email": profileemail,
                    "phone": profilephoneno,
                }
               }
        
        #write data to CRM ( create new contact in CRM)
        huboutput = hubspot.connect(HS_API_TOKEN).contacts.send(data)
        HS_CONTACT_ID_LIST.append([huboutput,profileurl])

✔️ Contact (id=1051) successfully created.
✔️ Contact (id=1101) successfully created.
✔️ Contact (id=1151) successfully created.
✔️ Contact (id=1201) successfully created.
✔️ Contact (id=1251) successfully created.
✔️ Contact (id=1301) successfully created.
✔️ Contact (id=1351) successfully created.
✔️ Contact (id=1401) successfully created.
✔️ Contact (id=1451) successfully created.
✔️ Contact (id=1501) successfully created.


## Output

### Display result - Create tasks for user and send email notification

In [21]:
HUB_CONTACT_URL = "https://app.hubspot.com/contacts/"+HUB_COMPANY_ID+"/contact/"

ENGAGEMENT_URL = "https://api.hubapi.com/engagements/v1/engagements"

QUERY_STRING = { "hapikey": HS_API_TOKEN }

#email parameter(s)
SEND_EMAIL_TO = ""

#timestamp delay to set due date for tasks
tstampobj = datetime.now() + timedelta(days=10)
tstamp = tstampobj.timestamp() * 1000

for i in HS_CONTACT_ID_LIST:
    hs = hubspot.connect(HS_API_TOKEN)
    contact_id = i[0]
    contact = hs.contacts.get(contact_id)
    contact_props = contact.get('properties')
    contact_linkedin_url = i[1]
    hs_contact_url = HUB_CONTACT_URL + contact_id
    payload = json.dumps({
        "engagement": {
            "active": 'true',
            "ownerId": "",
            "type": "TASK",
            "timestamp": tstamp
        },
        "associations": {
            "contactIds": [HS_CONTACT_ASSIGN_TO_ID],
            "companyIds": [],
            "dealIds": [],
            "ownerIds": [],
        },

        "metadata": {
            "body": "Hi there, you need to contact following user & task is already assigned to you.<br/>" + "Name :" +contact_props['firstname']+ contact_props['lastname'] + " Contact URL : " + hs_contact_url,
            "subject": "Task created for Contact ID :"+ contact_id,
            "status": "NOT_STARTED",
            "forObjectType": "CONTACT"
        }
    });
    headers = {
        'Content-Type': "application/json",
    }
    response = requests.request("POST", ENGAGEMENT_URL, data=payload, headers=headers, params=QUERY_STRING)

    if response.status_code == 200:
        print("Task created successfully for Contact ID :",contact_id)
        email_to = SEND_EMAIL_TO #to send the report
        subject = "LinkedIN Leads Alert"
        content = f"<p>Hi there,</p><br/><p>A new task has been created for you with contact ID :" + contact_id + "</p><br/><p>Actions : <a href="+hs_contact_url+">Open Hubspot Contact</a> | <a href="+contact_linkedin_url+">Open LinkedIN Profile</a></p>"

        naas.notification.send(email_to=email_to, subject=subject, html=content)
    else:
        print("Could not create the task for Contact ID :"+contact_id+", ask for on hello@naas.ai for more details")

Task created successfully for Contact ID : 1051
👌 💌 Email has been sent successfully !
Task created successfully for Contact ID : 1101
👌 💌 Email has been sent successfully !
Task created successfully for Contact ID : 1151
👌 💌 Email has been sent successfully !
Task created successfully for Contact ID : 1201
👌 💌 Email has been sent successfully !
Task created successfully for Contact ID : 1251
👌 💌 Email has been sent successfully !
Task created successfully for Contact ID : 1301
👌 💌 Email has been sent successfully !
Task created successfully for Contact ID : 1351
👌 💌 Email has been sent successfully !
Task created successfully for Contact ID : 1401
👌 💌 Email has been sent successfully !
Task created successfully for Contact ID : 1451
👌 💌 Email has been sent successfully !
Task created successfully for Contact ID : 1501
👌 💌 Email has been sent successfully !
